In [3]:
from flask import Flask, render_template, request, jsonify
from transformers import pipeline
from googletrans import Translator

c:\Users\srira\anaconda3\envs\deeplearning\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
app=Flask(__name__)


In [5]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=0)
text_generator = pipeline("text-generation", model="gpt2", device=0)
sentiment_analyser = pipeline("sentiment-analysis", device=0)
translator=Translator()

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [6]:
@app.route('/')
def home():
    return render_template('index.html')

In [7]:
@app.route('/process', methods=['POST'])
def process_text():
    # Validate JSON payload
    if not request.json:
        return jsonify({'error': 'Invalid or missing JSON payload'}), 400

    text = request.json.get('text')
    operation = request.json.get('operation')

    # Validate input text and operation
    if not text:
        return jsonify({'error': 'Text is required'}), 400
    if len(text.strip()) == 0 or len(text) > 1024:
        return jsonify({'error': 'Text must be non-empty and max 1024 characters'}), 400
    if not operation:
        return jsonify({'error': 'Operation is required'}), 400

    # Perform requested operation
    try:
        if operation == 'summarize':
            summary = summarizer(text, max_length=130, min_length=30)[0]['summary_text']
            return jsonify({'result': summary})
        elif operation == 'generate':
            generated = text_generator(text, max_length=100)[0]['generated_text']
            return jsonify({'result': generated})
        elif operation == 'sentiment':
            sentiment = sentiment_analyzer(text)[0]
            return jsonify({'result': f"Sentiment: {sentiment['label']}, Score: {sentiment['score']:.2f}"})
        elif operation == 'translate':
            translated = translator.translate(text, dest='en')
            return jsonify({'result': translated.text})
        else:
            return jsonify({'error': 'Invalid operation'}), 400
    except Exception as e:
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

c:\Users\srira\anaconda3\envs\deeplearning\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
